In [1]:

import pandas as pd
import numpy as np
import json
from sklearn.cluster import KMeans
import folium
import requests
%matplotlib inline
!conda install -c conda-forge folium=0.5.0 --yes
!pip install xlrd

import xlrd
import csv

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1 conda-forge --> 2019.9.11-py36_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
     |████████████████████████████████| 112kB 27.7MB/s eta 0:00:01
Libraries imported.


In [2]:
pwd

'/resources/labs/DP0701EN'

In [3]:
num_alo= pd.read_excel("num_alo3.xlsx")

In [4]:
num_alo=num_alo.rename(columns={'Âmbito Geográfico':'Denomination','Região':'Cities', 'Total':'Nº accommodations', 'Hotéis':'Hotels'})
num_alo.head(5)

,Denomination,Cities,Nº accommodations,Hotels
0,NUTS 2013,Portugal,6868,1400
1,NUTS I,Continente,4963,1242
2,NUTS II,Norte,1438,363
3,NUTS III,Alto Minho,242,44
4,Município,Arcos de Valdevez,36,4


In [5]:
num_alo= num_alo.fillna(0)
num_alo.head(5)

,Denomination,Cities,Nº accommodations,Hotels
0,NUTS 2013,Portugal,6868,1400
1,NUTS I,Continente,4963,1242
2,NUTS II,Norte,1438,363
3,NUTS III,Alto Minho,242,44
4,Município,Arcos de Valdevez,36,4


In [6]:
num_alo.dtypes

Denomination         object
Cities               object
Nº accommodations     int64
Hotels                int64
dtype: object

In [7]:
num_alo.astype({'Nº accommodations':'int','Hotels':'int'}).dtypes

Denomination         object
Cities               object
Nº accommodations     int64
Hotels                int64
dtype: object

In [8]:
Major=num_alo.sort_values('Hotels', ascending=False)
Major=Major.drop_duplicates(subset='Cities', keep='first')
df=Major[Major.Denomination == "Município"]
df.head(20)


,Denomination,Cities,Nº accommodations,Hotels
213,Município,Lisboa,576,189
38,Município,Porto,281,89
183,Município,Ourém,89,54
343,Município,Funchal,724,43
292,Município,Albufeira,209,42
212,Município,Cascais,82,27
315,Município,Ponta Delgada,93,24
17,Município,Braga,43,24
299,Município,Loulé,102,22
302,Município,Portimão,85,20


In [80]:
df2=Major[Major.Denomination == "NUTS III"]
df2.head(5)

,Denomination,Cities,Nº accommodations,Hotels
207,NUTS III,Área Metropolitana de Lisboa,908,276
30,NUTS III,Área Metropolitana do Porto,453,162
291,NUTS III,Algarve,817,158
176,NUTS III,Médio Tejo,189,78
340,NUTS III,Ilha da Madeira,1470,76


# Project - Compare the 20 Cities with more tourism in Portugal.

In the Major table I have Districts and Cities. So I will choose to this project the 20 cities with more Hotels as index to choose the Cities.

In [81]:
City= df.drop(df.columns[[0,2]], axis=1)
City=City.head(21)
City
Cidades=City.reset_index(drop=True, inplace=False)

In [83]:
Cidades.head(21)

,Cities,Hotels
0,Lisboa,189
1,Porto,89
2,Ourém,54
3,Funchal,43
4,Albufeira,42
5,Cascais,27
6,Ponta Delgada,24
7,Braga,24
8,Loulé,22
9,Portimão,20


# Finding the coordinates of the 20 cities

In [12]:
!conda install -c conda-forge geopy --yes 
!pip install geocoder
from geopy.geocoders import Nominatim
!pip install geopandas
!pip install geopy


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [84]:
cit= Cidades['Cities'].tolist()
cit

['Lisboa',
 'Porto',
 'Ourém',
 'Funchal',
 'Albufeira',
 'Cascais',
 'Ponta Delgada',
 'Braga',
 'Loulé',
 'Portimão',
 'Leiria',
 'Évora',
 'Lagos',
 'Coimbra',
 'Guimarães',
 'Viana do Castelo',
 'Figueira da Foz',
 'Matosinhos',
 'Viseu',
 'Faro',
 'Setúbal']

In [85]:
# Test Location
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Praça do Municipio, lisboa')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 38.708066, Longitude = -9.13913439292238


In [87]:
# I create a loop to fill the table Cidades with the cities coordenates, 
#so I need to create 2 more  columns

latitude_values = []
longitude_values = []
for n, row in Cidades.iterrows():
    location = locator.geocode(row['Cities'], 'Portugal')
    latitude_values.append(location.latitude)
    longitude_values.append(location.longitude)
    

Cidades['latitude'] = latitude_values
Cidades['longitude'] = longitude_values

In [90]:
Cidades

,Cities,Hotels,latitude,longitude
0,Lisboa,189,38.707751,-9.136592
1,Porto,89,41.149451,-8.610788
2,Ourém,54,39.687293,-8.567930
3,Funchal,43,32.649650,-16.908678
4,Albufeira,42,37.144524,-8.214588
5,Cascais,27,38.722400,-9.396909
6,Ponta Delgada,24,37.739434,-25.668362
7,Braga,24,41.551058,-8.428005
8,Loulé,22,37.214275,-8.086524
9,Portimão,20,37.194210,-8.585577


In [91]:
# Some coordinates are wrong, I will fixit. 
Cidades.iloc[4, Cidades.columns.get_loc('latitude'),]=37.089506 
Cidades.iloc[0, Cidades.columns.get_loc('latitude'),]=38.725147
Cidades.iloc[8, Cidades.columns.get_loc('latitude'),]=37.079227
Cidades.iloc[9, Cidades.columns.get_loc('latitude'),]=37.126283
Cidades.iloc[2, Cidades.columns.get_loc('latitude'),]=39.633666
Cidades.iloc[20, Cidades.columns.get_loc('latitude'),]=38.52341
Cidades.iloc[5, Cidades.columns.get_loc('latitude'),]=38.696812
Cidades.iloc[12, Cidades.columns.get_loc('latitude'),]=37.099555
Cidades.iloc[19, Cidades.columns.get_loc('latitude'),]=37.019869

Cidades.iloc[4, Cidades.columns.get_loc('longitude'),]=-8.246843
Cidades.iloc[0, Cidades.columns.get_loc('longitude'),]=-9.1449973
Cidades.iloc[8, Cidades.columns.get_loc('longitude'),]=-8.110704
Cidades.iloc[9, Cidades.columns.get_loc('longitude'),]=-8.538482
Cidades.iloc[2, Cidades.columns.get_loc('longitude'),]=-8.670023
Cidades.iloc[20, Cidades.columns.get_loc('longitude'),]=-8.894496
Cidades.iloc[5, Cidades.columns.get_loc('longitude'),]=-9.424695
Cidades.iloc[12, Cidades.columns.get_loc('longitude'),]=-8.676873
Cidades.iloc[19, Cidades.columns.get_loc('longitude'),]=-7.929246

Cidades= Cidades.drop([17], axis=0)


In [92]:
Cidades


,Cities,Hotels,latitude,longitude
0,Lisboa,189,38.725147,-9.144997
1,Porto,89,41.149451,-8.610788
2,Ourém,54,39.633666,-8.670023
3,Funchal,43,32.649650,-16.908678
4,Albufeira,42,37.089506,-8.246843
5,Cascais,27,38.696812,-9.424695
6,Ponta Delgada,24,37.739434,-25.668362
7,Braga,24,41.551058,-8.428005
8,Loulé,22,37.079227,-8.110704
9,Portimão,20,37.126283,-8.538482


# Portugal Map

In [93]:
latitude=38.707751
longitude=-7.00
#  Map of Portugal using latitude and longitude values
map_Portugal = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, citie in zip(Cidades['latitude'], Cidades['longitude'], Cidades['Cities']):
    label = '{}'.format(citie)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Portugal)  
    
map_Portugal

# Exploring the 20 Cities

In [20]:
import json
from pandas.io.json import json_normalize

In [94]:
Cidades.dtypes

Cities        object
Hotels         int64
latitude     float64
longitude    float64
dtype: object

In [95]:
CLIENT_ID = 'G0IMCUAWOZUTCEEX4ZCEKFEOSNR0F00QQDTHXSVQCTIUVNTV' 
CLIENT_SECRET = 'ZTE3WKRZGW20MEFSKP3U2DB2JE1B0OGA5MEA2BXREZ1TQIYN' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G0IMCUAWOZUTCEEX4ZCEKFEOSNR0F00QQDTHXSVQCTIUVNTV
CLIENT_SECRET:ZTE3WKRZGW20MEFSKP3U2DB2JE1B0OGA5MEA2BXREZ1TQIYN


In [96]:
# Test the coordenates
City_lat=Cidades.loc[12, 'latitude']
City_lon=Cidades.loc[12, 'longitude']
City=Cidades.loc[12, 'Cities']

print("The Location of {} are {}, {}.".format(City, City_lat, City_lon))

The Location of Lagos are 37.099555, -8.676873.


In [97]:
radius = 3000
LIMIT = 200

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, City_lat, City_lon, VERSION, radius, LIMIT)
results = requests.get(url).json()

In [98]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
venues = results['response']['groups'][0]['items']
    
City_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
City_venues =City_venues.loc[:, filtered_columns]

# filter the category for each row
City_venues['venue.categories'] = City_venues.apply(get_category_type, axis=1)

# clean columns
City_venues.columns = [col.split(".")[-1] for col in City_venues.columns]

City_venues.shape

(100, 4)

In [100]:
City_venues

,name,categories,lat,lng
0,Casa Mãe,Resort,37.103160,-8.675444
1,Pro Putting Garden,Mini Golf,37.097574,-8.673841
2,Ol' Bastard's,Fish & Chips Shop,37.100300,-8.672826
3,Dos Artistas,Portuguese Restaurant,37.100917,-8.673181
4,Goji,Lounge,37.101067,-8.674296
...,...,...,...,...
95,Restaurante Caniço,Seafood Restaurant,37.118430,-8.578679
96,Petisqueira A Oficina,Portuguese Restaurant,37.160397,-8.613044
97,Café na Ria,Bar,37.128912,-8.596323
98,Pepperino Pizza Napoletana,Italian Restaurant,37.129906,-8.594769


In [101]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        PT_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        PT_venues.columns = ['Cities', 'Cities Latitude', 'Cities Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(PT_venues)

In [102]:
PT_venues= getNearbyVenues(Cidades['Cities'], Cidades['latitude'], Cidades['longitude'])

Lisboa
Porto
Ourém
Funchal
Albufeira
Cascais
Ponta Delgada
Braga
Loulé
Portimão
Leiria
Évora
Lagos
Coimbra
Guimarães
Viana do Castelo
Figueira da Foz
Viseu
Faro
Setúbal


In [103]:
print (PT_venues.shape)
PT_venues.head(1500)

(1774, 7)


,Cities,Cities Latitude,Cities Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lisboa,38.725147,-9.144997,Hotel H10 Duque de Loulé,38.726364,-9.147198,Hotel
1,Lisboa,38.725147,-9.144997,Empanaderia El Pibe,38.723452,-9.143910,South American Restaurant
2,Lisboa,38.725147,-9.144997,O Forninho Saloio,38.722544,-9.145027,Portuguese Restaurant
3,Lisboa,38.725147,-9.144997,Rubro,38.722319,-9.146024,Spanish Restaurant
4,Lisboa,38.725147,-9.144997,Restaurante PSI,38.723359,-9.142014,Vegetarian / Vegan Restaurant
...,...,...,...,...,...,...,...
1495,Viseu,40.657471,-7.913866,Delícia Chocolateria,40.657115,-7.915473,Coffee Shop
1496,Viseu,40.657471,-7.913866,O Cortiço,40.659159,-7.911278,Restaurant
1497,Viseu,40.657471,-7.913866,Palato Wine Bar,40.659212,-7.911845,Wine Bar
1498,Viseu,40.657471,-7.913866,Dux Palace,40.656155,-7.917101,Portuguese Restaurant


In [104]:
PT_venues.groupby(['Cities']).size()

Cities
Albufeira           100
Braga               100
Cascais             100
Coimbra             100
Faro                 87
Figueira da Foz      42
Funchal             100
Guimarães           100
Lagos               100
Leiria               88
Lisboa              100
Loulé                48
Ourém                55
Ponta Delgada       100
Portimão            100
Porto               100
Setúbal             100
Viana do Castelo     78
Viseu               100
Évora                76
dtype: int64

In [105]:
PT=PT_venues
PT.head()

,Cities,Cities Latitude,Cities Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lisboa,38.725147,-9.144997,Hotel H10 Duque de Loulé,38.726364,-9.147198,Hotel
1,Lisboa,38.725147,-9.144997,Empanaderia El Pibe,38.723452,-9.143910,South American Restaurant
2,Lisboa,38.725147,-9.144997,O Forninho Saloio,38.722544,-9.145027,Portuguese Restaurant
3,Lisboa,38.725147,-9.144997,Rubro,38.722319,-9.146024,Spanish Restaurant
4,Lisboa,38.725147,-9.144997,Restaurante PSI,38.723359,-9.142014,Vegetarian / Vegan Restaurant


# Top Categories

In [106]:
# one hot encoding
PT_onehot = pd.get_dummies(PT[['Venue Category']], prefix="", prefix_sep="")

# add cities column back to dataframe
PT_onehot['Cities'] = PT['Cities'] 

# move Cities column to the first column
fixed_columns = [PT_onehot.columns[-1]] + list(PT_onehot.columns[:-1])
PT_onehot = PT_onehot[fixed_columns]

In [107]:
PT_onehot.head()

,Cities,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Lisboa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lisboa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lisboa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Lisboa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lisboa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [223]:
PT_grouped = PT_onehot.groupby('Cities').sum().reset_index()
PT_grouped.head()

,Cities,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Albufeira,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Braga,0,0,0,2,1,0,1,1,0,...,1,1,0,0,0,0,3,1,0,0
2,Cascais,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,Coimbra,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,Faro,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1


In [227]:
pd.options.display.max_columns = 180
PT_grouped.head()

,Cities,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Cable Car,Café,Campground,Candy Store,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dessert Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Lighthouse,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Resort,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Albufeira,1,1,0,0,0,0,0,0,0,1,0,1,10,0,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,1,0,0,0,0,1,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,7,0,0,0,1,1,0,0,5,0,0,0,0,0,3,0,4,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,13,1,0,3,7,0,1,0,0,0,0,1,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Braga,0,0,0,2,1,0,1,1,0,0,0,1,14,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,7,0,0,0,0,0,1,0,1,1,0,3,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,3,0,0,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,0,0,1,0,0,0,1,0,0,0,0,0,0,5,0,0,7,1,0,0,14,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,1,3,0,0,1,1,0,0,0,0,3,1,0,0
2,Cascais,0,0,1,0,1,0,0,0,0,1,0,3,3,1,5,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,2,0,0,0,1,6,0,0,0,3,2,0,0,4,1,0,0,0,2,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,2,1,0,8,2,0,1,9,0,0,0,0,0,0,1,0,7,0,0,3,0,0,0,0,0,0,0,0,0,0,3,1,1,1,2,0,0,0,0,0,0,0,0,1,1,0,0
3,Coimbra,0,0,0,1,0,0,0,0,0,0,0,4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,1,2,0,0,0,7,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,2,0,0,0,0,2,1,0,2,0,6,0,0,0,1,1,0,0,3,0,0,0,0,0,2,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,4,0,1,0,0,0,0,9,0,0,16,0,0,0,3,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,1,0,0,0,0,1,0,0
4,Faro,0,0,0,0,0,0,0,0,0,2,0,5,4,0,0,0,0,2,1,0,0,0,1,0,0,0,2,4,0,0,0,2,0,0,0,1,0,0,0,0,2,0,3,0,0,0,0,0,0,1,0,0,2,1,0,2,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,3,0,2,0,0,0,0,0,0,0,1,0,0,1,0,0,3,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,6,1,0,0,4,0,0,0,1,0,

In [233]:
PT_grouped['Portuguese Restaurant'] = PT_grouped['Restaurant'] + PT_grouped ['Portuguese Restaurant'] + PT_grouped['Mediterranean Restaurant']
PT_grouped=PT_grouped.drop(['Restaurant','Mediterranean Restaurant'], axis=1)

In [237]:
print (PT_grouped.shape)
PT_grouped.head()

(20, 163)


,Cities,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Cable Car,Café,Campground,Candy Store,Casino,Castle,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dessert Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Lighthouse,Lounge,Market,Mexican Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Resort,Road,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Albufeira,1,1,0,0,0,0,0,2,0,1,0,1,10,0,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,1,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,7,0,0,0,1,0,0,5,0,0,0,0,3,0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,46,1,0,3,0,1,0,0,0,0,1,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Braga,0,0,0,2,1,0,1,4,0,0,0,1,14,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,7,0,0,0,0,0,0,1,1,0,3,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,3,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,0,0,1,0,0,0,1,0,0,0,0,0,0,5,0,0,49,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,0,0,1,1,0,0,0,0,3,1,0,0
2,Cascais,0,0,1,0,1,0,0,4,0,1,0,3,3,1,5,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,2,0,0,0,1,6,0,0,0,3,0,0,4,0,0,0,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,2,1,0,38,2,0,1,0,0,0,0,0,0,1,0,7,0,0,3,0,0,0,0,0,0,0,0,0,0,3,1,1,2,0,0,0,0,0,0,0,0,1,1,0,0
3,Coimbra,0,0,0,1,0,0,0,1,0,0,0,4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,1,2,0,0,0,7,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,2,0,0,0,2,1,0,2,0,6,0,0,0,1,0,0,3,0,0,0,0,2,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,4,0,1,0,0,0,0,9,0,0,25,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,1,0,0,0,0,1,0,0
4,Faro,0,0,0,0,0,0,0,2,0,2,0,5,4,0,0,0,0,2,1,0,0,0,1,0,0,0,2,4,0,0,0,2,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,0,1,0,0,2,1,0,2,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,3,0,2,0,0,0,0,0,0,1,0,1,0,0,3,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,21,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,1,3,0,2,0,0,0,1,0,0,0,0,0,1,0,1


In [235]:
PT_grouped['Asian Restaurant']=PT_grouped['Asian Restaurant'] + PT_grouped['Chinese Restaurant'] + PT_grouped['Himalayan Restaurant'] + PT_grouped['Indian Restaurant'] + PT_grouped['Japanese Restaurant'] + PT_grouped['Sushi Restaurant']
PT_grouped=PT_grouped.drop(['Chinese Restaurant','Himalayan Restaurant', 'Indian Restaurant', 'Japanese Restaurant' , 'Sushi Restaurant' ], axis=1)
print (PT_grouped.shape)

(20, 163)


In [239]:
PT_grouped['Fast Food Restaurant']=PT_grouped['Burger Joint'] + PT_grouped['BBQ Joint'] + PT_grouped['Fried Chicken Joint'] + PT_grouped ['Pizza Place'] + PT_grouped['Fast Food Restaurant']
PT_grouped=PT_grouped.drop(['Burger Joint','Fried Chicken Joint', 'Pizza Place', 'BBQ Joint' ], axis=1)
print (PT_grouped.shape)

(20, 159)


In [256]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [257]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cities']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
PT_venues_sorted = pd.DataFrame(columns=columns)
PT_venues_sorted['Cities'] = PT_grouped['Cities']

for ind in np.arange(PT_grouped.shape[0]):
    PT_venues_sorted.iloc[ind, 1:] = return_most_common_venues(PT_grouped.iloc[ind, :], num_top_venues)



In [258]:
PT_venues_sorted.head(21)

,Cities,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albufeira,Portuguese Restaurant,Bar,Fast Food Restaurant,Beach,Hotel,Italian Restaurant,Seafood Restaurant,Lounge,Cocktail Bar,Resort
1,Braga,Portuguese Restaurant,Bar,Café,Plaza,Fast Food Restaurant,Asian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Tapas Restaurant
2,Cascais,Portuguese Restaurant,Seafood Restaurant,Fast Food Restaurant,Hotel,Beach,Asian Restaurant,Italian Restaurant,Steakhouse,Snack Place,Bar
3,Coimbra,Portuguese Restaurant,Plaza,Café,Hotel,Bar,Tapas Restaurant,Fast Food Restaurant,Bakery,Park,Italian Restaurant
4,Faro,Portuguese Restaurant,Fast Food Restaurant,Bakery,Bar,Hostel,Coffee Shop,Sporting Goods Shop,Lounge,Supermarket,Café
5,Figueira da Foz,Fast Food Restaurant,Portuguese Restaurant,Hotel,Café,Beach,Bar,Coffee Shop,Ice Cream Shop,Tapas Restaurant,Harbor / Marina
6,Funchal,Portuguese Restaurant,Hotel,Fast Food Restaurant,Café,Bar,Bakery,Asian Restaurant,Plaza,Lounge,Seafood Restaurant
7,Guimarães,Portuguese Restaurant,Fast Food Restaurant,Café,Bar,Hotel,Plaza,Asian Restaurant,Electronics Store,Historic Site,Tea Room
8,Lagos,Portuguese Restaurant,Fast Food Restaurant,Beach,Resort,Hotel,Seafood Restaurant,Grocery Store,Tapas Restaurant,Supermarket,Bar
9,Leiria,Portuguese Restaurant,Fast Food Restaurant,Bar,Café,Plaza,Electronics Store,Supermarket,Asian Restaurant,Tapas Restaurant,Park


# Cluster the Cities

In [259]:
# K-means
PT_KM=PT_grouped.drop('Cities',1)
k_means=KMeans(
                init="k-means++",
                n_clusters=3,
                n_init=12)
k_means.fit(PT_KM)
k_means_label=k_means.labels_

In [260]:
k_means_label

array([0, 0, 0, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0],
      dtype=int32)

In [261]:
PT_venues_sorted.insert(0,'Cluster', k_means_label )


In [262]:
PT_venues_sorted.head()


,Cluster,Cities,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Albufeira,Portuguese Restaurant,Bar,Fast Food Restaurant,Beach,Hotel,Italian Restaurant,Seafood Restaurant,Lounge,Cocktail Bar,Resort
1,0,Braga,Portuguese Restaurant,Bar,Café,Plaza,Fast Food Restaurant,Asian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Tapas Restaurant
2,0,Cascais,Portuguese Restaurant,Seafood Restaurant,Fast Food Restaurant,Hotel,Beach,Asian Restaurant,Italian Restaurant,Steakhouse,Snack Place,Bar
3,1,Coimbra,Portuguese Restaurant,Plaza,Café,Hotel,Bar,Tapas Restaurant,Fast Food Restaurant,Bakery,Park,Italian Restaurant
4,2,Faro,Portuguese Restaurant,Fast Food Restaurant,Bakery,Bar,Hostel,Coffee Shop,Sporting Goods Shop,Lounge,Supermarket,Café


In [263]:
PT_venues_sorted.reset_index
PT_venues_sorted.set_index('Cities')
PT_venues_sorted.head(20)

,Cluster,Cities,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Albufeira,Portuguese Restaurant,Bar,Fast Food Restaurant,Beach,Hotel,Italian Restaurant,Seafood Restaurant,Lounge,Cocktail Bar,Resort
1,0,Braga,Portuguese Restaurant,Bar,Café,Plaza,Fast Food Restaurant,Asian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Tapas Restaurant
2,0,Cascais,Portuguese Restaurant,Seafood Restaurant,Fast Food Restaurant,Hotel,Beach,Asian Restaurant,Italian Restaurant,Steakhouse,Snack Place,Bar
3,1,Coimbra,Portuguese Restaurant,Plaza,Café,Hotel,Bar,Tapas Restaurant,Fast Food Restaurant,Bakery,Park,Italian Restaurant
4,2,Faro,Portuguese Restaurant,Fast Food Restaurant,Bakery,Bar,Hostel,Coffee Shop,Sporting Goods Shop,Lounge,Supermarket,Café
5,2,Figueira da Foz,Fast Food Restaurant,Portuguese Restaurant,Hotel,Café,Beach,Bar,Coffee Shop,Ice Cream Shop,Tapas Restaurant,Harbor / Marina
6,1,Funchal,Portuguese Restaurant,Hotel,Fast Food Restaurant,Café,Bar,Bakery,Asian Restaurant,Plaza,Lounge,Seafood Restaurant
7,1,Guimarães,Portuguese Restaurant,Fast Food Restaurant,Café,Bar,Hotel,Plaza,Asian Restaurant,Electronics Store,Historic Site,Tea Room
8,2,Lagos,Portuguese Restaurant,Fast Food Restaurant,Beach,Resort,Hotel,Seafood Restaurant,Grocery Store,Tapas Restaurant,Supermarket,Bar
9,2,Leiria,Portuguese Restaurant,Fast Food Restaurant,Bar,Café,Plaza,Electronics Store,Supermarket,Asian Restaurant,Tapas Restaurant,Park


In [264]:
Cities_Label = pd.DataFrame(columns= None)

Cities_Label ['Cities'] = Cidades ['Cities']
Cities_Label ['latitude'] = Cidades ['latitude']
Cities_Label ['longitude'] = Cidades ['longitude']
Cities_Label.reset_index
Cities_Label.set_index ('Cities')

,latitude,longitude
Cities,,
Lisboa,38.725147,-9.144997
Porto,41.149451,-8.610788
Ourém,39.633666,-8.670023
Funchal,32.649650,-16.908678
Albufeira,37.089506,-8.246843
Cascais,38.696812,-9.424695
Ponta Delgada,37.739434,-25.668362
Braga,41.551058,-8.428005
Loulé,37.079227,-8.110704


In [265]:
Cities_Label = Cities_Label.join(PT_venues_sorted.set_index('Cities'), on='Cities')


In [266]:
Cities_Label

,Cities,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lisboa,38.725147,-9.144997,1,Portuguese Restaurant,Hotel,Fast Food Restaurant,Asian Restaurant,Scenic Lookout,Garden,Plaza,Coffee Shop,Vegetarian / Vegan Restaurant,Park
1,Porto,41.149451,-8.610788,1,Portuguese Restaurant,Hostel,Bar,Fast Food Restaurant,Ice Cream Shop,Plaza,Wine Bar,Nightclub,Coffee Shop,Cocktail Bar
2,Ourém,39.633666,-8.670023,1,Portuguese Restaurant,Hotel,Café,Trail,Fast Food Restaurant,Bar,Breakfast Spot,Bakery,Bed & Breakfast,Park
3,Funchal,32.649650,-16.908678,1,Portuguese Restaurant,Hotel,Fast Food Restaurant,Café,Bar,Bakery,Asian Restaurant,Plaza,Lounge,Seafood Restaurant
4,Albufeira,37.089506,-8.246843,0,Portuguese Restaurant,Bar,Fast Food Restaurant,Beach,Hotel,Italian Restaurant,Seafood Restaurant,Lounge,Cocktail Bar,Resort
5,Cascais,38.696812,-9.424695,0,Portuguese Restaurant,Seafood Restaurant,Fast Food Restaurant,Hotel,Beach,Asian Restaurant,Italian Restaurant,Steakhouse,Snack Place,Bar
6,Ponta Delgada,37.739434,-25.668362,1,Portuguese Restaurant,Hotel,Fast Food Restaurant,Café,Bar,Bakery,Plaza,Seafood Restaurant,Coffee Shop,Museum
7,Braga,41.551058,-8.428005,0,Portuguese Restaurant,Bar,Café,Plaza,Fast Food Restaurant,Asian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Tapas Restaurant
8,Loulé,37.079227,-8.110704,1,Portuguese Restaurant,Seafood Restaurant,Lounge,Hotel,Beach,Café,Coffee Shop,Mexican Restaurant,Nightclub,Italian Restaurant
9,Portimão,37.126283,-8.538482,0,Portuguese Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Beach,Café,Bar,Ice Cream Shop,Lounge,Nightclub


# Mapping the  clusters cities of Portugal

In [268]:
import matplotlib.cm as cm
import matplotlib.colors as colors
cl=Cities_Label['Cluster'].astype(int)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)


# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cl in zip(Cities_Label['latitude'], Cities_Label['longitude'], Cities_Label['Cities'], cl,):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cl), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cl-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

# Review the clusters cities

In [269]:
Cities_Label.sort_values(by=['Cluster'])

,Cities,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Portimão,37.126283,-8.538482,0,Portuguese Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Beach,Café,Bar,Ice Cream Shop,Lounge,Nightclub
4,Albufeira,37.089506,-8.246843,0,Portuguese Restaurant,Bar,Fast Food Restaurant,Beach,Hotel,Italian Restaurant,Seafood Restaurant,Lounge,Cocktail Bar,Resort
5,Cascais,38.696812,-9.424695,0,Portuguese Restaurant,Seafood Restaurant,Fast Food Restaurant,Hotel,Beach,Asian Restaurant,Italian Restaurant,Steakhouse,Snack Place,Bar
7,Braga,41.551058,-8.428005,0,Portuguese Restaurant,Bar,Café,Plaza,Fast Food Restaurant,Asian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Coffee Shop,Tapas Restaurant
11,Évora,38.570774,-7.909281,0,Portuguese Restaurant,Hotel,Historic Site,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Bar,Bus Station,Café,Bed & Breakfast
18,Viseu,40.657471,-7.913866,1,Portuguese Restaurant,Fast Food Restaurant,Bar,Café,Hotel,Coffee Shop,Electronics Store,Bed & Breakfast,Plaza,Park
15,Viana do Castelo,41.694867,-8.831088,1,Portuguese Restaurant,Fast Food Restaurant,Bakery,Hotel,Bar,Coffee Shop,Supermarket,Beach,Plaza,Park
14,Guimarães,41.441768,-8.295571,1,Portuguese Restaurant,Fast Food Restaurant,Café,Bar,Hotel,Plaza,Asian Restaurant,Electronics Store,Historic Site,Tea Room
13,Coimbra,40.210980,-8.429206,1,Portuguese Restaurant,Plaza,Café,Hotel,Bar,Tapas Restaurant,Fast Food Restaurant,Bakery,Park,Italian Restaurant
0,Lisboa,38.725147,-9.144997,1,Portuguese Restaurant,Hotel,Fast Food Restaurant,Asian Restaurant,Scenic Lookout,Garden,Plaza,Coffee Shop,Vegetarian / Vegan Restaurant,Park


In [289]:
new_pt =pd.DataFrame(columns= None)
new_pt = PT_grouped
new_pt = new_pt.drop(['Cities'], axis=1)
new_pt.head()

,Cluster,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burrito Place,Bus Station,Cable Car,Café,Campground,Candy Store,Casino,Castle,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dessert Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Lighthouse,Lounge,Market,Mexican Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Resort,Road,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,1,1,0,0,0,0,0,2,0,0,1,10,0,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,1,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,8,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,7,0,0,0,1,0,0,5,0,0,0,0,3,0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,46,1,0,3,0,1,0,0,0,0,1,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,2,1,0,1,4,0,0,1,14,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,7,0,0,0,0,0,0,1,1,0,3,0,0,0,0,1,1,2,0,0,0,0,0,4,0,0,1,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,3,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,0,0,1,0,0,0,1,0,0,0,0,0,5,0,0,49,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,0,0,1,1,0,0,0,0,3,1,0,0
2,0,0,0,1,0,1,0,0,4,0,0,3,3,1,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,6,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,2,0,0,0,1,6,0,0,0,3,0,0,4,0,0,0,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,2,1,0,38,2,0,1,0,0,0,0,0,0,1,0,7,0,0,3,0,0,0,0,0,0,0,0,0,0,3,1,1,2,0,0,0,0,0,0,0,0,1,1,0,0
3,1,0,0,0,1,0,0,0,1,0,0,4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,7,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,2,0,0,4,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,2,0,0,0,2,1,0,2,0,6,0,0,0,1,0,0,3,0,0,0,0,2,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,4,0,1,0,0,0,9,0,0,25,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,1,0,0,0,0,1,0,0
4,2,0,0,0,0,0,0,0,2,0,0,5,4,0,0,0,0,2,1,0,0,0,1,0,0,0,2,0,0,0,2,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,0,1,0,0,2,1,0,20,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,3,0,2,0,0,0,0,0,0,1,0,1,0,0,3,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,1,3,0,2,0,0,0,1,0,0,0,0,0,1,0,1


In [300]:
new=new_pt.groupby('Cluster').sum().reset_index()
new

,Cluster,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burrito Place,Bus Station,Cable Car,Café,Campground,Candy Store,Casino,Castle,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dessert Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Lighthouse,Lounge,Market,Mexican Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Resort,Road,Rock Club,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,1.0,1.0,1.0,2.0,2.0,0.0,1.0,15.0,0.0,1.0,7.0,33.0,1.0,17.0,0.0,4.0,0.0,1.0,1.0,4.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,5.0,8.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,33.0,2.0,1.0,2.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,9.0,0.0,1.0,1.0,1.0,35.0,0.0,1.0,0.0,9.0,0.0,0.0,13.0,0.0,0.0,0.0,2.0,6.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,0.0,2.0,9.0,1.0,0.0,230.0,4.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,20.0,0.0,0.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,1.0,7.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,0.0
1,1,1.0,0.0,0.0,4.0,0.0,1.0,0.0,21.0,1.0,0.0,26.0,46.0,0.0,7.0,0.0,9.0,3.0,5.0,1.0,2.0,0.0,2.0,2.0,2.0,4.0,8.0,0.0,0.0,3.0,46.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,4.0,9.0,24.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,0.0,10.0,0.0,1.0,71.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,8.0,1.0,5.0,0.0,1.0,3.0,2.0,5.0,6.0,1.0,0.0,0.0,10.0,4.0,0.0,11.0,1.0,73.0,3.0,0.0,1.0,14.0,3.0,0.0,13.0,1.0,1.0,0.0,0.0,15.0,2.0,2.0,0.0,2.0,3.0,0.0,4.0,6.0,4.0,1.0,12.0,2.0,1.0,0.0,1.0,0.0,3.0,17.0,2.0,3.0,1.0,1.0,2.0,34.0,1.0,0.0,282.0,3.0,1.0,1.0,2.0,0.0,1.0,2.0,8.0,1.0,5.0,1.0,15.0,10.0,0.0,7.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,6.0,7.0,1.0,11.0,7.0,1.0,1.0,7.0,2.0,0.0,2.0,2.0,7.0,9.0,0.0,0.0
2,2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10.0,0.0,0.0,11.0,21.0,0.0,12.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,2.0,0.0,1.0,2.0,4.0,1.0,0.0,0.0,23.0,0.0,1.0,1.0,3.0,0.0,1.0,0.0,3.0,4.0,10.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,1.0,0.0,79.0,6.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,7.0,0.0,15.0,0.0,1.0,0.0,6.0,1.0,1.0,5.0,0.0,1.0,1.0,0.0,6.0,5.0,3.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,91.0,2.0,0.0,7.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,0.0,17.0,4.0,1.0,4.0,0.0,1.0,0.0,0.